In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
from pathlib import Path
from glob import glob
from subprocess import check_call, CalledProcessError

import numpy as np
import yaml

import matplotlib.pyplot as plt

import cson_forge


Query the blueprint catalog to find the run blueprint matching the specified model and grid. This provides the path to the simulation output directory.


Find all model output files matching the pattern. These files contain the time-series data for the variable of interest.

In [4]:
domains_file = "domains-bm-scaling.yml"

In [ ]:
# Read domains file and extract domain keys
# Resolve domains file path (relative to notebook directory)
domains_path = Path(domains_file)
if not domains_path.is_absolute():
    domains_path = Path.cwd() / domains_path

# Load domains YAML file
with open(domains_path, 'r') as f:
    domains_data = yaml.safe_load(f) or {}

# Extract domain keys (main keys in the YAML file)
domain_keys = list(domains_data.keys())

print(f"Loaded {len(domain_keys)} domain(s) from {domains_path}")
print(f"Domain keys: {domain_keys}")


Loaded 5 domain(s) from /Users/mclong/codes/cson-forge/workflows/computing-benchmarks/domains-bm-scaling.yml
Domain keys: ['ccs-12km_20procs', 'ccs-12km_70procs', 'ccs-12km_280procs', 'ccs-12km_352procs', 'ccs-12km_1120procs']


In [ ]:
# Parse SLURM Job ID from log file
# Function is now available from cson_forge.parsers
from cson_forge.parsers import parse_slurm_job_id

# Example usage:
# log_file = Path("path/to/logfile.txt")
# job_id = parse_slurm_job_id(log_file)
# print(f"SLURM Job ID: {job_id}")



Load the model grid from the blueprint's grid YAML file. The grid provides coordinate information (lon_rho, lat_rho) and masking for proper visualization.

In [10]:
df = cson_forge.catalog.blueprint.load(stage="run")
# Blueprint names are {model_name}_{grid_name}_{n_procs}procs
# Domain keys are {grid_name}_{n_procs}procs
# So we need to check if blueprint_name ends with any domain_key
df = df.loc[df.blueprint_name.str.endswith(tuple(domain_keys))]

if df.empty:
    raise ValueError(
        f"No blueprints found: blueprint_name='{domain_keys}' "
        f"at stage='run'"
    )

df

,model_name,grid_name,blueprint_name,description,start_time,end_time,blueprint_path,grid_yaml_path,stage
0,cson_roms-marbl_v0.1_ccs-12km,20procs,cson_roms-marbl_v0.1_ccs-12km_20procs,California Current System,2024-01-01T00:00:00,2024-01-02T00:00:00,/Users/mclong/codes/cson-forge/cson_forge/blue...,/Users/mclong/codes/cson-forge/cson_forge/blue...,run
1,cson_roms-marbl_v0.1_ccs-12km,280procs,cson_roms-marbl_v0.1_ccs-12km_280procs,California Current System,2024-01-01T00:00:00,2024-01-02T00:00:00,/Users/mclong/codes/cson-forge/cson_forge/blue...,/Users/mclong/codes/cson-forge/cson_forge/blue...,run
2,cson_roms-marbl_v0.1_ccs-12km,352procs,cson_roms-marbl_v0.1_ccs-12km_352procs,California Current System,2024-01-01T00:00:00,2024-01-02T00:00:00,/Users/mclong/codes/cson-forge/cson_forge/blue...,/Users/mclong/codes/cson-forge/cson_forge/blue...,run
3,cson_roms-marbl_v0.1_ccs-12km,70procs,cson_roms-marbl_v0.1_ccs-12km_70procs,California Current System,2024-01-01T00:00:00,2024-01-02T00:00:00,/Users/mclong/codes/cson-forge/cson_forge/blue...,/Users/mclong/codes/cson-forge/cson_forge/blue...,run


In [ ]:
job_ids = []
for i in range(len(df)):
    blueprint_path = df.blueprint_path.iloc[i]
    casename = blueprint_path.stem.replace("B_", "") # get less janky
    simulation_dir = Path(cson_forge.config.paths.scratch / casename)
    log_file = simulation_dir / "logs" / f"{casename}.out"
    if not log_file.exists():
        print(f"Log file {log_file} does not exist")
        continue
    job_ids.append(cson_forge.parsers.parse_slurm_job_id(log_file))

core_hours_list = []
for job_id in job_ids:
    core_hours = cson_forge.parsers.sacct_summary(job_id)["ntasks_core_hours"]
    if core_hours is None:
        print(f"Job {job_id} has no core hours")
        continue
    core_hours_list.append(core_hours)

core_hours_list



Blueprint path: /Users/mclong/codes/cson-forge/cson_forge/blueprints/NERSC_perlmutter/cson_roms-marbl_v0.1_ccs-12km_20procs/B_cson_roms-marbl_v0.1_ccs-12km_20procs_run_20240101-20240102.yml
Simulation directory: /Users/mclong/cson-forge-data/cson-forge-run/cson_roms-marbl_v0.1_ccs-12km_20procs_run_20240101-20240102
